In [6]:
%%writefile AlexNet_EBLL.py

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
import torch.utils.data as data
import torchvision
from torchvision import datasets, models, transforms
import sys
import pdb



class AutoEncoder(torch.nn.Module):
    def __init__(self, x_dim, h1_dim):
        super(AutoEncoder, self).__init__()
        self.encode = nn.Sequential( 
            #fc layers for the encoder
            nn.Linear(x_dim , h1_dim),
            nn.Sigmoid())
        
        #fc layers for the decoder
        self.decode = nn.Sequential( 
            nn.Linear(h1_dim, x_dim),

        )
        

    def forward(self, x):
        h = self.encode(x) 
        x_recon = self.decode(h)        
        return x_recon

Overwriting AlexNet_EBLL.py


In [7]:
%%writefile -a AlexNet_EBLL.py

class autoencoders(torch.nn.Module):
    def __init__(self,autoencoder):
        super(autoencoders, self).__init__()
        self.add_module('0',autoencoder.encode)
          
    def forward(self, x):
        outputs=[]
       
        for name, module in self._modules.items():
                outputs.append(module(x))
          
        return outputs

Appending to AlexNet_EBLL.py


In [8]:
%%writefile -a AlexNet_EBLL.py




class AlexNet_ENCODER(nn.Module):
    def __init__(self, alexnet,dim=100):
        super(AlexNet_ENCODER, self).__init__()
        
        self.add_module('features',alexnet.features)
        self.add_module('autoencoder',AutoEncoder(256 * 6 * 6,dim))
        self.add_module('classifier',alexnet.classifier)
      
       
        
    def forward(self, x):
        
        sub_index=0
        
        encoder=False
        last_layer=False
        for name, module in self._modules.items():
                
                for namex, modulex in module._modules.items():
                    if last_layer:
                        out = modulex(x)
                    else:
                        x= modulex(x)
                    if name=='classifier' and namex=='5':
                        last_layer=True
                if name=='autoencoder':
                    encoder_output=x
 
                    
                #for reshaping the fully connected layers
                #need to be changed for 
                if sub_index==0:
                  
                    x = x.view(x.size(0), 256 * 6 * 6)
                    encoder_input=x.data.clone()
                    #forward to the encoder
                sub_index+=1
        #pdb.set_trace()        
        return out,encoder_input,encoder_output
    

Appending to AlexNet_EBLL.py


In [9]:
%%writefile -a AlexNet_EBLL.py


class AlexNet_EBLL(nn.Module):
    def __init__(self, module,autoencoder):
        super(AlexNet_EBLL, self).__init__()
        self.add_module('features',module.features)
        self.add_module('autoencoders',autoencoders(autoencoder))
        self.add_module('classifier',module.classifier)
      
       
        
    def forward(self, x):
        
        sub_index=0
        last_layer=False
        for name, module in self._modules.items():
                if name =='autoencoders':
                    codes=module(x)
                else:    
                    for namex, modulex in module._modules.items():

                        if last_layer:
                            outputs.append(modulex(x))
                        else:

                             x = modulex(x)

                        if name=='classifier' and namex=='5':

                            last_layer=True
                            outputs=[]

                #for reshaping the fully connected layers
                #need to be changed for 
                if sub_index==0:
                    
                    x = x.view(x.size(0), 256 * 6 * 6)
                sub_index+=1
        #pdb.set_trace()        
        return outputs,codes
    

Appending to AlexNet_EBLL.py
